In [1]:
begin
	using Pkg
	Pkg.activate("..")
	# Pkg.instantiate()
	
	using PlutoUI
	using CairoMakie
	using LinearAlgebra
	using Luxor
	using BenchmarkTools
	using Printf
	using ImageMorphology
	using LoopVectorization
	using CUDA
	using CUDA:i32
	using Test
end

  Activating project at `c:\Users\wenbl13\Desktop\project-distance-transforms-wenbo-2`


In [2]:
boolean_indicator(f) = @. ifelse(f == 0, 1f10, 0f0)

boolean_indicator (generic function with 1 method)

In [3]:
function boolean_indicator(img::BitArray)
	f = similar(img, Float32)
	@turbo for i in CartesianIndices(f)
           @inbounds f[i] = img[i] ? 0f0 : 1f10
    end
	return f
end

boolean_indicator (generic function with 2 methods)

## Multi-Thded and Nonmulti-Thded

## 1D

In [41]:
begin
	function DT1helperA!(f, j)
		# i == -1 && j != -1
		temp=1
		while j>0
			@inbounds f[j]=temp^2
			j-=1
			temp+=1
		end
	end
	function DT1helperB!(f, i)
		# i != -1 && j == -1
		temp=1
		l = length(f)
		while i<=l
			@inbounds f[i]=temp^2
			i+=1
			temp+=1
		end
	end
	function DT1helperC!(f, i, j)
		# i != -1 && j != -1
		temp=1
		while(i<=j)
			@inbounds f[i]=f[j]=temp^2
			temp+=1
			i+=1
			j-=1
		end
	end 
	function DT1Wenbo!(f)
		pointerA = 1
		l = length(f)
		while pointerA <= l
			while pointerA <= l && @inbounds f[pointerA] == 0
				pointerA+=1
			end
			pointerB = pointerA
			while pointerB <= l && @inbounds f[pointerB] == 1f10
				pointerB+=1
			end
			if pointerB > length(f)
				if pointerA == 1
					return
				else
					DT1helperB!(f, pointerA)
				end
			else
				if pointerA == 1
					DT1helperA!(f, pointerB-1)
				else
					DT1helperC!(f, pointerA, pointerB-1)
				end
			end
			pointerA=pointerB
		end
	end
	function DT1Wenbo(f)
		f = boolean_indicator(f)
		DT1Wenbo!(f)
		return f
	end
end

DT1Wenbo (generic function with 1 method)

## 2D

In [42]:
begin
	function encode(leftD, rightf)
		if rightf == 1f10
			return -leftD
		end
		idx = 0
		while(rightf>1)
			rightf  /=10
			idx+=1 
		end
		return -leftD-idx/10-rightf/10
	end
	function decode(curr)	
		curr *= -10   				
		temp = Int(floor(curr))		
		curr -= temp 				
		if curr == 0
			return 1f10
		end
		temp %= 10
		while temp > 0
			temp -= 1
			curr*=10
		end
		return round(curr)
	end
	function DT2Helper!(f)
		l = length(f)
		pointerA = 1
		while pointerA<=l && @inbounds f[pointerA] <= 1
			pointerA += 1
		end
		p = 0
		while pointerA<=l
			@inbounds curr = f[pointerA]
			prev = curr
			temp = min(pointerA-1, p+1)
			p = 0
			while (0 < temp)
				@inbounds fi = f[pointerA-temp]
				fi = fi < 0 ? decode(fi) : fi
				newDistance = muladd(temp, temp, fi)
				if newDistance < curr
					curr = newDistance
					p = temp
				end
				temp -= 1
			end
			temp = 1
			templ = length(f) - pointerA
			while (temp <= templ && muladd(temp, temp, -curr) < 0)
				@inbounds curr = min(curr, muladd(temp, temp, f[pointerA+temp]))
				temp += 1
			end
			@inbounds f[pointerA] = encode(curr, prev)
			# end
			pointerA+=1
			while pointerA<=l && @inbounds f[pointerA] <= 1
				pointerA += 1
			end
		end
		i = 0
		while i<l
			i+=1
			f[i] = floor(abs(f[i]))
		end
	end
	function DT2WenboA!(f)
		for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2WenboB!(f)
		Threads.@threads for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		Threads.@threads for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2Wenbo(f)
		f = boolean_indicator(f)
		DT2tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT2WenboB! : DT2WenboA!
		DT2tf!(f)
		return f
	end
end

DT2Wenbo (generic function with 1 method)

## 3D

In [6]:
begin
	function DT3WenboA!(f)
		for i in axes(f, 3)
		    @inbounds DT2WenboA!(@view(f[:, :, i]))
		end
		for i in CartesianIndices(f[:,:,1])
			@inbounds DT2Helper!(@view(f[i, :]))
		end
	end 
	function DT3WenboB!(f)
		Threads.@threads for i in axes(f, 3)
		    @inbounds DT2WenboB!(@view(f[:, :, i]))
		end
		Threads.@threads for i in CartesianIndices(f[:,:,1])
			@inbounds DT2Helper!(@view(f[i, :]))
		end
	end
	function DT3Wenbo(f)
		f = boolean_indicator(f)
		DT3tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT3WenboB! : DT3WenboA!
		DT3tf!(f)
		return f
	end
end

DT3Wenbo (generic function with 1 method)

## Timing: Wenbo vs ImageMorphology

In [5]:
euclideanImageMorphology(img::BitArray) = distance_transform(feature_transform(img))

euclideanImageMorphology (generic function with 1 method)

In [18]:
begin
	img1DsmallB = Bool.(rand([0, 1], 15))
	img1DbigB = Bool.(rand([0, 1], 4000))
	img1DB = Bool.(rand([0, 1], 200))
	img2DB = Bool.(rand([0, 1], 200, 800))
	img3DB = Bool.(rand([0, 1], 200, 400, 600))
	img2D4kB = Bool.(rand([0, 1], 3840, 2160))
	img1DsmallG = CuArray(img1DsmallB)
	img1DbigG = CuArray(img1DbigB)
	img1DG = CuArray(img1DB)
	img2DG = CuArray(img2DB)
	img3DG = CuArray(img3DB)
	img2D4kG = CuArray(img2D4kB)
	"Test inputs created."
end

"Test inputs created."

### 1D

In [9]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT1Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.0
				"failed"
				break
			end
		end
	end
	"1D: 100 test cases passed!"
end

"1D: 100 test cases passed!"

In [10]:
@benchmark DT1Wenbo($img1DB)

BenchmarkTools.Trial: 10000 samples with 401 evaluations.
 Range (min … max):  235.411 ns … 30.528 μs  ┊ GC (min … max):  0.00% … 98.27%
 Time  (median):     457.606 ns              ┊ GC (median):     0.00%
 Time  (mean ± σ):   488.570 ns ±  1.298 μs  ┊ GC (mean ± σ):  13.21% ±  4.91%

     ▃▄                 ▂▂▂▂▅█                                  
  ▃▇▇██▇▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▄██████▇▅▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  235 ns          Histogram: frequency by time          777 ns <

 Memory estimate: 896 bytes, allocs estimate: 1.

In [11]:
@benchmark euclideanImageMorphology($img1DB)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.340 μs …  1.163 ms  ┊ GC (min … max):  0.00% … 99.71%
 Time  (median):     2.430 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.657 μs ± 25.790 μs  ┊ GC (mean ± σ):  17.17% ±  2.44%

  ▆█▅▂ ▃▆▇▅▂▁                 ▂▃▂▁▁▂▃▂▁                      ▂
  ███████████████▇▇▇▇▇▆▅▅▅▃▃▅████████████▇▇▆▇▇▆▆▆▅▅▆▆▅▆▅▅▅▅▅ █
  1.34 μs      Histogram: log(frequency) by time     11.5 μs <

 Memory estimate: 8.97 KiB, allocs estimate: 6.

### 2D

In [12]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n, n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT2Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.
				"failed"
				break
			end
		end
	end
	"2D: 100 test cases passed!"
end

"2D: 100 test cases passed!"

In [13]:
@benchmark DT2Wenbo($img2DB)

BenchmarkTools.Trial: 3221 samples with 1 evaluation.
 Range (min … max):  1.236 ms …  14.492 ms  ┊ GC (min … max): 0.00% … 89.20%
 Time  (median):     1.505 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.549 ms ± 580.793 μs  ┊ GC (mean ± σ):  2.41% ±  5.58%

                      ▃ ▂▆█▇▅▂                                 
  ▁▁▁▁▁▁▁▁▁▂▂▃▃▃▃▃▃▄▆██████████▇▆▄▃▂▃▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  1.24 ms         Histogram: frequency by time        1.88 ms <

 Memory estimate: 625.05 KiB, allocs estimate: 2.

In [14]:
@benchmark euclideanImageMorphology($img2DB)

BenchmarkTools.Trial: 938 samples with 1 evaluation.
 Range (min … max):  4.411 ms … 19.059 ms  ┊ GC (min … max): 0.00% … 58.78%
 Time  (median):     5.033 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.322 ms ±  1.490 ms  ┊ GC (mean ± σ):  4.42% ± 10.23%

   ▅███▆▄▂                                                    
  ▇████████▅▄▁▄▄▆▄▄▁▄▆▅▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▅▆▆▁▇ █
  4.41 ms      Histogram: log(frequency) by time     13.8 ms <

 Memory estimate: 3.75 MiB, allocs estimate: 10.

### 3D

In [15]:
let
	n = 200
	for i = 1: 100
		f = Bool.(rand([0, 1], n, n, n))
		rslt1 = euclideanImageMorphology(f);
		rslt1 .^ 2
		rslt2 = DT3Wenbo(f)
		for i in CartesianIndices(rslt1)
			if rslt1[i] - rslt2[i] != 0.0
				"failed"
				break
			end
		end
	end
	"3D: 100 test cases passed!"
end

"3D: 100 test cases passed!"

In [16]:
@benchmark DT3Wenbo($img3DB)

BenchmarkTools.Trial: 9 samples with 1 evaluation.
 Range (min … max):  552.447 ms … 755.521 ms  ┊ GC (min … max): 0.00% … 27.79%
 Time  (median):     577.809 ms               ┊ GC (median):    0.12%
 Time  (mean ± σ):   595.573 ms ±  62.847 ms  ┊ GC (mean ± σ):  5.41% ±  9.19%

  █  ▁▁  ▁   ▁▁   ▁                                           ▁  
  █▁▁██▁▁█▁▁▁██▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  552 ms           Histogram: frequency by time          756 ms <

 Memory estimate: 183.41 MiB, allocs estimate: 4.

In [17]:
@benchmark euclideanImageMorphology($img3DB)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.493 s …   2.629 s  ┊ GC (min … max): 0.13% … 4.41%
 Time  (median):     2.561 s              ┊ GC (median):    2.33%
 Time  (mean ± σ):   2.561 s ± 96.022 ms  ┊ GC (mean ± σ):  2.33% ± 3.03%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.49 s         Histogram: frequency by time        2.63 s <

 Memory estimate: 1.44 GiB, allocs estimate: 480009.

# Rewriting Wenbo!() for GPU Compatibility

In [6]:
function _boolean_indicator_GPU!(out, f)
	i = threadIdx().x + (blockIdx().x - 1) * blockDim().x
	@inbounds out[i] = f[i] ? 0f0 : 1f10
	return
end

_boolean_indicator_GPU! (generic function with 1 method)

In [7]:
begin
	b_i_GPU_kernels = []
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 1))
	push!(b_i_GPU_kernels, kernel)
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 2}(undef, 1, 1), CuArray{Bool, 2}(undef, 1, 1))
	push!(b_i_GPU_kernels, kernel)
	kernel = @cuda launch=false _boolean_indicator_GPU!(CuArray{Float32, 3}(undef, 1, 1, 1), CuArray{Bool, 3}(undef, 1, 1, 1))
	push!(b_i_GPU_kernels, kernel)
	config_threads = launch_configuration(kernel.fun).threads
	"Created boolean_indicator_GPU kernels."
end

"Created boolean_indicator_GPU kernels."

In [8]:
function boolean_indicator_GPU(f)
		# input = CuArray(img)
		output = similar(f, Float32)
		threads = min(length(f), config_threads)
		blocks = cld(length(f), threads)
		b_i_GPU_kernels[ndims(f)](output, f; threads, blocks)
		return output
end

boolean_indicator_GPU (generic function with 1 method)

In [9]:
@benchmark boolean_indicator($img2DB)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   36.000 μs …  11.151 ms  ┊ GC (min … max):  0.00% … 98.27%
 Time  (median):     162.200 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   182.997 μs ± 467.589 μs  ┊ GC (mean ± σ):  16.70% ±  6.52%

  ▇▃▃▂           ▂▅▇▆█▆▅▄▃▃▃▂▂▂▂▂▁▁▁ ▁                          ▂
  █████▇▇▅▅▃▄▄▄▃▅████████████████████████▆▆▆▆▆▆▅▅▄▅▅▆▆▆▅▆▅▅▇▆▅▆ █
  36 μs         Histogram: log(frequency) by time        433 μs <

 Memory estimate: 625.05 KiB, allocs estimate: 2.

In [38]:
begin
	function leftEnd!(out,f)
		i = 1
		while i <= length(f) && @inbounds !f[i]
			i += 1
		end
		out[i] = 0f0
		i -= 1
		ct = 1
		while i >= 0
			@inbounds out[i] = ct^2
			ct += 1
			i -= 1
		end
		return
	end
	kernel1 = @cuda launch=false leftEnd!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 0))

	function rightEnd!(out,f)
		i = length(f)
		while i >= 0 && @inbounds !f[i]
			i -= 1
		end
		i += 1
		ct = 1
		while i <= length(f)
			@inbounds out[i] = ct^2
			ct += 1
			i += 1
		end
		return
	end
	kernel2 = @cuda launch=false rightEnd!(CuArray{Float32, 1}(undef, 1), CuArray{Bool, 1}(undef, 0))

	function temp2!(out, f, ct)
		i = threadIdx().x
		block_start = (i-1)*ct+1
		block_end = i*ct
		prev= -1
		j = block_start
		while j <= block_end
			if @inbounds f[j]
				prev = j
				break
			end
			j += 1
		end
		if prev == -1
			return
		end
		j = prev+1
		while j <= block_end
			if @inbounds f[j]
				ct = 1
				out[j] = 0f0
				while prev+2*ct-j <= 0
					@inbounds out[j-ct] = out[prev+ct] = ct^2
					ct += 1
				end
				prev = j
			end
			j += 1
		end
		while j <= length(f)
			if @inbounds f[j]
				ct = 1
				out[j] = 0f0
				while prev+2*ct-j <= 0
					@inbounds out[j-ct] = out[prev+ct] = ct^2
					ct += 1
				end
				return
			end
			j += 1
		end
		return
	end

	kernel3 = @cuda launch=false temp2!(CuArray{Float32, 1}(undef, 0), CuArray{Bool, 1}(undef, 0),0)
	function DT1Wenbo_GPU(f)
		l = length(f)
		output = CuArray{Float32, 1}(undef, l)
		ct = cld(l, config_threads)
		threads = fld(l,ct)
		kernel3(output, f, ct; threads)
		kernel1(output, f)
		kernel2(output, f)
		# f = boolean_indicator_GPU(f)
		# DT1Wenbo_GPU!(f)
		return output
	end
end

DT1Wenbo_GPU (generic function with 1 method)

In [29]:
begin
	function DT1Wenbo_GPU!(out, f)
		pointerA = 1
		l = length(f)
		while pointerA <= l
			while pointerA <= l && @inbounds f[pointerA]
				out[pointerA] = 0f0
				pointerA+=1
			end
			pointerB = pointerA
			while pointerB <= l && @inbounds !f[pointerB]
				pointerB+=1
			end
			if pointerB > length(f)
				if pointerA == 1
					i=length(f)
					while i>0
						@inbounds out[i]=1f10
						i-=1
					end
				else
					i = pointerA 
					out[pointerA-1] = 0f0
					temp=1
					l = length(f)
					while i<=l
						@inbounds out[i]=temp^2
						i+=1
						temp+=1
					end
				end
			else
				if pointerA == 1
					j = pointerB-1
					out[pointerB] = 0f0
					temp=1
					while j>0
						@inbounds out[j]=temp^2
						j-=1
						temp+=1
					end
				else
					out[pointerA-1] = 0f0
					out[pointerB] = 0f0
					i = pointerA
					j = pointerB-1
					temp=1
					while i<=j
						@inbounds out[i]=out[j]=temp^2
						temp+=1
						i+=1
						j-=1
					end
				end
			end
			pointerA=pointerB
		end
	end

	kernel4 = @cuda launch=false DT1Wenbo_GPU!(CuArray{Float32, 1}(undef, 0), CuArray{Bool, 1}(undef, 0))

	function DT1Wenbo_GPU2(f)
		output = CuArray{Float32, 1}(undef, length(f))
		kernel4(output, f)
		return output
	end
end

DT1Wenbo_GPU2 (generic function with 1 method)

In [47]:
CUDA.reclaim()

In [30]:
rslt_gpu = Array(DT1Wenbo_GPU2(img1DbigG))

4000-element Vector{Float32}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0

In [26]:
rslt_cpu = DT1Wenbo(img1DbigB)

4000-element Vector{Float32}:
 0.0
 0.0
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 1.0
 0.0
 ⋮
 1.0
 0.0
 0.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0

In [28]:
n = 7

for i = n-5: n+5
    print("$(i)\t")
end
println()

for i = n-5: n+5
    print("$(img1DbigB[i])\t")
end
println()

for i = n-5: n+5
    print("$(rslt_cpu[i])\t")
end
println()

for i = n-5: n+5
    print("$(rslt_gpu[i])\t")
end

2	3	4	5	6	7	8	9	10	11	12	
true	false	true	true	true	true	true	false	true	false	true	
0.0	1.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	1.0	0.0	
0.0	1.0	0.0	1.0	0.0	2.3694275e-38	0.0	1.0	0.0	1.0	0.0	

In [31]:
for i = 1: length(img1DG)
    if rslt_cpu[i] != rslt_gpu[i]
        println("i = $i, cpu: $(rslt_cpu[i]), gpu: $(rslt_gpu[i])")
    end
end

In [32]:
@test Array(DT1Wenbo_GPU2(img1DbigG)) == DT1Wenbo(img1DbigB)

Test Passed
  Expression: Array(DT1Wenbo_GPU2(img1DbigG)) == DT1Wenbo(img1DbigB)
   Evaluated: Float32[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0] == Float32[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0  …  0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0]

In [77]:
begin
	function encode_GPU!(out, leftD, rightf)
		out[1] = -leftD
		if rightf == 1f10
			return nothing
		end
		idx = 0
		while(rightf>1)
			rightf  /=10
			idx+=1 
		end
        out[1] += -idx/10-rightf/10
		return nothing
	end
	encode_kernel = @cuda launch=false encode_GPU!(CuArray{Float32, 1}(undef, 1),0f0,0f0)

	function decode_GPU!(out, curr)	
		curr *= -10   				
		temp = Int(floor(curr))		
		curr -= temp 				
		if curr == 0
			out[1] = 1f10
			return nothing
		end
		temp %= 10
		while temp > 0
			temp -= 1
			curr*=10
		end
		out[1] = round(curr)
		return nothing
	end
	decode_kernel = @cuda launch=false decode_GPU!(CuArray{Float32, 1}(undef, 1),0f0)

	function DT2Helper_GPU!(f)
		l = length(f)
		pointerA = 1
		while pointerA<=l && @inbounds f[pointerA] <= 1
			pointerA += 1
		end
		p = 0
		while pointerA<=l
			@inbounds curr = f[pointerA]
			prev = curr
			temp = min(pointerA-1, p+1)
			p = 0
			while (0 < temp)
				@inbounds fi = f[pointerA-temp]
				fi = fi < 0 ? decode(fi) : fi
				newDistance = muladd(temp, temp, fi)
				if newDistance < curr
					curr = newDistance
					p = temp
				end
				temp -= 1
			end
			temp = 1
			templ = length(f) - pointerA
			while (temp <= templ && muladd(temp, temp, -curr) < 0)
				@inbounds curr = min(curr, muladd(temp, temp, f[pointerA+temp]))
				temp += 1
			end
			@inbounds f[pointerA] = encode(curr, prev)
			# end
			pointerA+=1
			while pointerA<=l && @inbounds f[pointerA] <= 1
				pointerA += 1
			end
		end
		i = 0
		while i<l
			i+=1
			f[i] = floor(abs(f[i]))
		end
	end
	function DT2WenboA!(f)
		for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2WenboB!(f)
		Threads.@threads for i in axes(f, 1)
			@inbounds DT1Wenbo!(@view(f[i, :]))
		end
		Threads.@threads for i in axes(f, 2)
			@inbounds DT2Helper!(@view(f[:,i]))
		end
	end
	function DT2Wenbo(f)
		f = boolean_indicator(f)
		DT2tf! = length(f) > 2700 && Threads.nthreads()>1 ?  DT2WenboB! : DT2WenboA!
		DT2tf!(f)
		return f
	end
end

DT2Wenbo (generic function with 1 method)

In [85]:
encode_GPU(13,25)

-13.224999f0

In [88]:
@benchmark decode(-13.224999f0)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  0.900 ns … 30.500 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.000 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.090 ns ±  0.992 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▄       █       ▅       ▂                                  ▁
  █▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█ █
  0.9 ns       Histogram: log(frequency) by time      1.6 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [86]:
@benchmark decode_kernel(temptemp,-13.224999f0)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.360 μs …  1.445 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.710 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.272 μs ± 22.292 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▅▇█▇▆▆▅▇█▇▇▇▅▄▃▂▂▁ ▁                                       ▃
  █████████████████████▇██▇▇█▇█▇▇▇▇▇▇▇▆▆▇▇██▇▇▇▇█▇▆▇▅▅▅▅▅▆▅▅ █
  1.36 μs      Histogram: log(frequency) by time     3.89 μs <

 Memory estimate: 720 bytes, allocs estimate: 18.